Ideas for improvement:
- Change hashing method so that the 5000 most common tokens are unique and all others are zero
- Change random_tweets to be more random, and remove foreign languages, etc.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import pandas as pd
import numpy as np

#local packages
import twitter
import text_processing

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

np.random.seed(152)

C:\Users\silve\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [39]:
#import tweet ids from hatespeech-master, filter for sexist tweets, and convert to a list
tweet_ids = pd.read_csv('hatespeech-master/NAACL_SRW_2016.csv', header=None)
tweet_ids = list(tweet_ids[tweet_ids[1] == 'sexism'][0])

sexist_tweets = twitter.get_statuses(tweet_ids)
sexist_tweet_text = [tweet['text'] for tweet in sexist_tweets]

In [50]:
#Now add a list of random tweets from random_tweets.csv
random_tweets = list(pd.read_csv('random_tweets2.csv', header=None)[1])

#Randomly select tweets from random_tweets so there are the same number as in sexist_tweets
random_tweets = np.random.choice(random_tweets, len(sexist_tweet_text), replace=False).tolist()

#Create a list of markers for sexist tweets:
#0 = not sexist; 1 = sexist
is_sexist = [1]*len(sexist_tweets) + [0]*len(random_tweets)

#Concatenate the sexist and random tweets
tweet_text = sexist_tweet_text + random_tweets

In [75]:
vocab_size = 10000
embedding_vector_length = 32

#Convert words to integer hashes
encoded_text = text_preprocessing.simple_hash(tweet_text, hashspace_size=vocab_size)
#Pad word sequences so they're all the same length
encoded_text = pad_sequences(encoded_text, padding='post')

max_tweet_length = max([len(tw) for tw in encoded_text])

In [76]:
#Split data into training and validation sets

#Use 25% of values as validation
n = len(encoded_text)
validation_indices = np.random.choice(n, size=(n // 4), replace=False)
training_indices = np.setdiff1d(np.arange(n), validation_indices)

X_train = np.array([encoded_text[i] for i in training_indices])
y_train = np.array([is_sexist[i] for i in training_indices])
X_test = np.array([encoded_text[i] for i in validation_indices])
y_test = np.array([is_sexist[i] for i in validation_indices])

In [77]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_tweet_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=32)

Train on 4442 samples, validate on 1480 samples
Epoch 1/2
4442/4442 [==============================] - 14s 3ms/step - loss: 0.6939 - acc: 0.4975 - val_loss: 0.6932 - val_acc: 0.5020
Epoch 2/2
4442/4442 [==============================] - 14s 3ms/step - loss: 0.6935 - acc: 0.4881 - val_loss: 0.6932 - val_acc: 0.4980


In [54]:
model.evaluate(X_test, y_test, verbose=0)

[0.6932087453635963, 0.5]

In [45]:
#Save trained model
model.save('model.h5')

In [11]:
def predict(text):
    if type(text) is str:
        text = [text]
    encoded = twitter.create_hash_indices(text, vocab_size)
    encoded = pad_sequences(encoded, maxlen=max_tweet_length, padding='post')
    return model.predict_classes(encoded)

In [78]:
predictions = model.predict_classes(encoded_text)

In [84]:
pd.Series(sexist_tweet_text).to_csv('sexist_tweets.csv')